# VisuaLIME example

This brief introduction show you how to generate a visual explanation for the classification of an image by a deep learning computer vision model.

## Load an image-classification model

For this example, we're using a relatively small pre-trained model provided as part of the Tensorflow deep learning library.

If you haven't installed Tensorflow in your current Python environment, uncomment and run the following line:

In [ ]:
#!pip install tensorflow-cpu

Then, we can load the modell and the corresponding preprocessing function:

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [ ]:
model = MobileNetV2()

Since LIME is a black box explanation method, it does not "know" anything about how to call the model to produce predictions. Instead, we need to provide a function that simply takes an array of images and returns the corresponding outputs:


In [ ]:
def predict_fn(image):
    return model.predict(preprocess_input(image))

## Load an image

We'll load an image hosted on the internet as part of the [XAI Demonstrator project](https://github.com/xai-demonstrator/xai-demonstrator). Instead, you can also load an image from your harddrive or from a different URL.

In [ ]:
from urllib.request import urlopen

import numpy as np
from PIL import Image

In [ ]:
full_image = Image.open(urlopen("https://storage.googleapis.com/xai-demo-assets/visual-inspection/images/table.jpg"))
full_image

We'll just select a single object:

In [ ]:
img = full_image.crop((766, 90, 990, 314))
img

VisuaLIME takes in images as Numpy arrays:

In [ ]:
image = np.array(img)
image.shape

Note that the image is 224 by 224 pixels, which is exactly the size our `model` expects. In general, it is advisable to compute explanations on the same scale as the model's input. So in case your image is larger or smaller than the size expected by the model, rescale it prior to passing it to the VisuaLIME algorithm.

Let's see whether we can predict what's in the image:

In [ ]:
prediction = predict_fn(image[None,:,:,:])
prediction.shape

We see that the output contains one classification result of length 1000. Each of the 1000 entries corresponds to the likelihood that the image belongs to that particular class.

Let's see what the model sees in the picture:

In [ ]:
np.argmax(prediction, axis=1)

So it's class 759. We can decode this either by looking up the ImageNet categories or using the provided decoder function:

In [ ]:
from tensorflow.keras.applications.mobilenet import decode_predictions
decode_predictions(prediction, top=1)

Great, so the model correctly identifies the camera in the image. But what exactly does it look at?

## Compute an explanation

To find out, import the two main functions of `visualime`:

In [ ]:
from visualime.explain import explain_classification, render_explanation

First, we'll compute the explanation:

In [ ]:
segment_mask, segment_weights = explain_classification(image=image, predict_fn=predict_fn, num_of_samples=128)

Then, we can generate the visual output:

In [ ]:
render_explanation(image, segment_mask, segment_weights, positive="green", negative="red", coverage=0.05)

So it seems like the lens is a strong factor in influencing the model's decision.